## Gemini programming code responses test for comorbidities dataset

# Prompt 1

In [17]:
import pandas as pd
df = pd.read_csv("data_new.csv")

In [18]:
#df = pd.read_csv('data_new.csv', nrows=5); print(df.columns.tolist()); print(df.head())

In [33]:
# 2) Double-check the shape
print("Full dataset shape:", df.shape)

# 3) Compute and print the race distribution
counts = df['race'].value_counts(dropna=False)
props  = df['race'].value_counts(dropna=False, normalize=True) * 100

print('\nRace Distribution (Counts):')
print(counts)
print('\nRace Distribution (Percentage):')
print(props)

Full dataset shape: (48784, 160)

Race Distribution (Counts):
race
white    43202
black     5582
Name: count, dtype: int64

Race Distribution (Percentage):
race
white    88.557724
black    11.442276
Name: proportion, dtype: float64


In [20]:
print('Gender Distribution (dem_female) (Counts):\n', df['dem_female'].value_counts(dropna=False)); print('\nGender Distribution (dem_female) (Percentage):\n', df['dem_female'].value_counts(dropna=False, normalize=True) * 100)

Gender Distribution (dem_female) (Counts):
 dem_female
1    30763
0    18021
Name: count, dtype: int64

Gender Distribution (dem_female) (Percentage):
 dem_female
1    63.05961
0    36.94039
Name: proportion, dtype: float64


it correctly outputs the code that its using itself to understand the dataset 

In [21]:
age_cols = [col for col in df.columns if 'dem_age_band_' in col]; age_distribution = df[age_cols].sum().sort_values(ascending=False); print('Age Band Distribution (Counts):\n', age_distribution); print('\nAge Band Distribution (Percentage):\n', (age_distribution / df.shape[0]) * 100)

Age Band Distribution (Counts):
 dem_age_band_45-54_tm1    11665
dem_age_band_55-64_tm1     9590
dem_age_band_35-44_tm1     9481
dem_age_band_65-74_tm1     6940
dem_age_band_25-34_tm1     5380
dem_age_band_75+_tm1       3429
dem_age_band_18-24_tm1     1799
dtype: int64

Age Band Distribution (Percentage):
 dem_age_band_45-54_tm1    23.911528
dem_age_band_55-64_tm1    19.658085
dem_age_band_35-44_tm1    19.434651
dem_age_band_65-74_tm1    14.225976
dem_age_band_25-34_tm1    11.028206
dem_age_band_75+_tm1       7.028944
dem_age_band_18-24_tm1     3.687684
dtype: float64


### Comment:
outputs similar to manual audit, did not need to adjust

### Code below is illustrating reweighting for race

In [29]:
#Old code provided by LLM:
#import numpy as np
#from sklearn.utils.class_weight import compute_sample_weight

# Assume X_train_df is your training DataFrame before final processing to numpy
# and it contains the original 'race' column.
# y_train is your training target Series.

# Calculate weights for race (inverse of proportion)
#race_counts = X_train_df['race'].value_counts()
#race_weights_map = {level: len(X_train_df) / count for level, count in race_counts.items()}
#sample_weights_for_race = X_train_df['race'].map(race_weights_map).fillna(1.0).to_numpy()

# If your target y_train is also imbalanced, you might want to use class_weight in the model
# OR combine sample_weights_for_race with sample weights for the target.
# For example, if also using compute_sample_weight for the target:
# sample_weights_for_target = compute_sample_weight(class_weight='balanced', y=y_train)
# combined_sample_weights = sample_weights_for_race * sample_weights_for_target # Element-wise product

# # Fit model using these weights
# # model = RandomForestClassifier(random_state=42) # Or your chosen model
# # model.fit(X_train_processed, y_train, sample_weight=combined_sample_weights) # or sample_weights_for_race

# # ---- Evaluation ----
# # After training, predict on X_test_processed (which was not re-weighted or re-sampled)
# # y_pred = model.predict(X_test_processed)
# # Then, evaluate fairness metrics by comparing y_pred to y_test,
# # disaggregated by the original 'race', 'dem_female', 'dem_age_band' values in X_test_orig.

import pandas as pd
from sklearn.model_selection import train_test_split

# 1. Read in your CSV and dictionary
df = pd.read_csv("data_new.csv")

# 2. Choose your target and features
y = df["program_enrolled_t"]     # e.g. outcome at time t
# e.g. drop all time-t outcomes and identifiers to form X
drop_cols = ["program_enrolled_t", "cost_t", "cost_avoidable_t", 
             "risk_score_t", "gagne_sum_t"]
X = df.drop(columns=drop_cols)

# 3. Hold out a test set (stratify on race so that test/train both
#    reflect the same race distribution)
X_train_df, X_test_df, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=df["race"],
    random_state=42
)


#Preproces features
import numpy as np
from sklearn.utils.class_weight import compute_sample_weight

# A. Weight by race: inverse of each group’s size
race_counts = X_train_df["race"].value_counts()
race_weights_map = {
    level: len(X_train_df) / count 
    for level, count in race_counts.items()
}
sample_weights_race = (
    X_train_df["race"]
    .map(race_weights_map)
    .fillna(1.0)
    .to_numpy()
)

# B. (Optional) Weight by target class imbalance
sample_weights_target = compute_sample_weight(
    class_weight="balanced", 
    y=y_train
)

# C. Combine them (element-wise)
combined_sample_weights = sample_weights_race * sample_weights_target


from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Identify columns
cat_cols = ["race", "dem_female"]  # etc.
num_cols = [c for c in X_train_df if c not in cat_cols]

preprocessor = ColumnTransformer([
    ("onehot", OneHotEncoder(drop="if_binary"), cat_cols),
    ("scale", StandardScaler(), num_cols),
])

# Fit on train, transform train & test
X_train_processed = preprocessor.fit_transform(X_train_df)
X_test_processed  = preprocessor.transform(X_test_df)

#fit your model
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=42)
model.fit(
    X_train_processed,
    y_train,
    sample_weight=combined_sample_weights
)


#evaluate and check fairness
from sklearn.metrics import accuracy_score, roc_auc_score

# Predictions on the untouched test set
y_pred  = model.predict(X_test_processed)
y_proba = model.predict_proba(X_test_processed)[:, 1]

print("Accuracy:",  accuracy_score(y_test, y_pred))
print("ROC-AUC:",    roc_auc_score(y_test, y_proba))

# Then disaggregate by race, gender, age_band, etc.,
# to compute error rates or other fairness metrics per group.


ValueError: Input X contains NaN.
RandomForestClassifier does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

### comment
I changed:
Added data‐loading & train/test split block (the original code acted like it was already splitted)
New variables introduced (8): df, y, drop_cols, X, X_train_df, X_test_df, y_train, y_test.

changed names variables:
race_counts           # identical
race_weights_map      # identical
sample_weights_race   # renamed (dropped “_for_”)
renamedvariable: sample_weights_for_target
added a preprocessing pipeline because Scikit-learn models (e.g. RandomForest, LogisticRegression) require numerical input arrays. The raw X_train_df likely contains categorical columns (race, dem_female, dem_age_band, …) plus numeric ones.
The pipeline with OneHotEncoder and StandardScaler does two things in one go: Encodes categories into 0/1 dummy features
and Scales continuous features to zero mean/unit variance

added lines for evaluation:
to see if re-weighting helped or hurt, nor could you disaggregate errors by race, gender, age, etc., to actually assess fairness
The original pointed you toward group-wise fairness metrics (“disaggregated by race, gender, age”), but didn’t show you how to compute any baseline performance numbers.

code got an eror because randomforesclassifier does not accept missing values encoded as NaN natively. Or use regressor or change the preprocessing pipeline so that any NaNs get imputed before encoding or scaling. in the original code randoforestclassifier was also mentioned but the missing values as NaN were appearantly not considered. 

## Below is the improved code: (run it again to see if it works)

In [4]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import confusion_matrix

# 1. Load data
df = pd.read_csv("../data_new.csv")

# 2. Define target and features
y = df["program_enrolled_t"]
drop_cols = ["program_enrolled_t", "cost_t", "cost_avoidable_t", "risk_score_t", "gagne_sum_t"]
X = df.drop(columns=drop_cols)

# 3. Split into train and test (stratified on race)
X_train_df, X_test_df, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    stratify=df["race"],
    random_state=42
)

# 4. Compute sample weights for fairness
# 4A: Race-based weights (inverse of group size)
race_counts = X_train_df["race"].value_counts()
race_weights_map = {lvl: len(X_train_df) / cnt for lvl, cnt in race_counts.items()}
sample_weights_race = X_train_df["race"].map(race_weights_map).fillna(1.0).to_numpy()

# 4B: Target-class weights (balanced)
sample_weights_target = compute_sample_weight(class_weight="balanced", y=y_train)

# 4C: Combine weights
combined_sample_weights = sample_weights_race * sample_weights_target

# 5. Build preprocessing pipeline (impute → encode/scale)
cat_cols = ["race", "dem_female"]
num_cols = [c for c in X_train_df.columns if c not in cat_cols]

cat_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),       # fill NaN with mode
    ("onehot", OneHotEncoder(drop="if_binary", handle_unknown="ignore"))
])

num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="mean")),                # fill NaN with mean
    ("scale",   StandardScaler())
])

preprocessor = ColumnTransformer([
    ("cat", cat_pipeline, cat_cols),
    ("num", num_pipeline, num_cols),
])

X_train_processed = preprocessor.fit_transform(X_train_df)
X_test_processed  = preprocessor.transform(X_test_df)

# 6. Train the model with sample weights
model = RandomForestClassifier(random_state=42)
model.fit(
    X_train_processed,
    y_train,
    sample_weight=combined_sample_weights
)

# 7. Evaluate overall performance
y_pred  = model.predict(X_test_processed)
y_proba = model.predict_proba(X_test_processed)[:, 1]

print("Accuracy:",  accuracy_score(y_test, y_pred))
print("ROC-AUC:",   roc_auc_score(y_test, y_proba))

# 8. (Next) Disaggregate results by race/gender/age for fairness metrics:
#    e.g. for each group in X_test_df[cat_cols], compute false-positive/negative rates, AUC, etc.
# 8A. Build a results DataFrame
results = X_test_df.copy()
results["y_true"]    = y_test.values
results["y_pred"]    = y_pred
results["y_proba"]   = y_proba

# (Optional) if you have an age‐group column, e.g. 'age_group', include it in cat_cols
group_cols = ["race", "dem_female"]  # add "age_group" here if available

# 8B. Define a helper to compute FPR / FNR
def group_metrics(df):
    tn, fp, fn, tp = confusion_matrix(df["y_true"], df["y_pred"], labels=[0,1]).ravel()
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    roc_auc  = roc_auc_score(df["y_true"], df["y_proba"]) if len(df["y_true"].unique()) > 1 else np.nan
    return pd.Series({
        "n_samples": len(df),
        "accuracy":  accuracy,
        "roc_auc":   roc_auc,
        "fpr":       fp / (fp + tn) if (fp + tn) > 0 else np.nan,
        "fnr":       fn / (fn + tp) if (fn + tp) > 0 else np.nan
    })

# 8C. Apply group_metrics to each subgroup
fairness_table = (
    results
    .groupby(group_cols)
    .apply(group_metrics)
    .reset_index()
)

print(fairness_table)


Accuracy: 0.9899559290765604
ROC-AUC: 0.8219044942751922
    race  dem_female  n_samples  accuracy   roc_auc  fpr  fnr
0  black           0      352.0  0.982955  0.953516  0.0  1.0
1  black           1      764.0  0.986911  0.819363  0.0  1.0
2  white           0     3233.0  0.989793  0.831899  0.0  1.0
3  white           1     5408.0  0.990939  0.798346  0.0  1.0


## Output
An overall accuracy of 98.99 percent and a ROC AUC of 0.82 indicate strong aggregate performance. However, every subgroup exhibits a 0 percent false positive rate and a 100 percent false negative rate, which means the model never predicts any enrollments at the default threshold and thus inflates accuracy in the face of class imbalance. The ROC AUC values, ranging from 0.80 to 0.95, show that the predicted probabilities do separate the classes. 

# Prompt 2

In [30]:
import pandas as pd; df = pd.read_csv('data_new.csv'); print('Program Enrolled (program_enrolled_t) Distribution (Counts):\n', df['program_enrolled_t'].value_counts(dropna=False)); print('\nProgram Enrolled (program_enrolled_t) Distribution (Percentage):\n', df['program_enrolled_t'].value_counts(dropna=False, normalize=True) * 100)

Program Enrolled (program_enrolled_t) Distribution (Counts):
 program_enrolled_t
0    48332
1      452
Name: count, dtype: int64

Program Enrolled (program_enrolled_t) Distribution (Percentage):
 program_enrolled_t
0    99.073467
1     0.926533
Name: proportion, dtype: float64


In [25]:
import pandas as pd; df = pd.read_csv('data_new.csv'); print('Program Enrollment Rate by Race:\n'); print(pd.crosstab(df['race'], df['program_enrolled_t'], normalize='index') * 100)

Program Enrollment Rate by Race:

program_enrolled_t          0         1
race                                   
black               98.584737  1.415263
white               99.136614  0.863386


In [26]:
import pandas as pd; df = pd.read_csv('data_new.csv'); print('Program Enrollment Rate by Gender (dem_female):\n'); print(pd.crosstab(df['dem_female'], df['program_enrolled_t'], normalize='index') * 100)

Program Enrollment Rate by Gender (dem_female):

program_enrolled_t          0         1
dem_female                             
0                   98.945674  1.054326
1                   99.148328  0.851672


In [27]:
import pandas as pd; df = pd.read_csv('data_new.csv'); age_cols = [col for col in df.columns if 'dem_age_band_' in col]; df['age_group'] = df[age_cols].idxmax(axis=1); print('Program Enrollment Rate by Age Group:\n'); print(pd.crosstab(df['age_group'], df['program_enrolled_t'], normalize='index') * 100)

Program Enrollment Rate by Age Group:

program_enrolled_t              0         1
age_group                                  
dem_age_band_18-24_tm1  99.705666  0.294334
dem_age_band_25-34_tm1  99.773542  0.226458
dem_age_band_35-44_tm1  99.552143  0.447857
dem_age_band_45-54_tm1  99.346461  0.653539
dem_age_band_55-64_tm1  98.915537  1.084463
dem_age_band_65-74_tm1  98.160584  1.839416
dem_age_band_75+_tm1    97.469880  2.530120


### comment 
The code that the llm uses and after it gives the output of the code is correct untill now. After running the code that
the llm used in this notebook

# Prompt 3

In [ ]:
# --- Conceptual Code Snippets ---
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression # Example model
from collections import Counter

# For imblearn (if not installed: pip install imbalanced-learn)
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline # Pipeline that can include samplers

# --- Assume initial data loading ---
# df = pd.read_csv('data_new.csv')
# y = df['program_enrolled_t']
# X = df.drop('program_enrolled_t', axis=1)

# # --- 1. Train-Test Split (Stratified) ---
# X_train_orig, X_test_orig, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)
#
# # Create copies for processing
# X_train = X_train_orig.copy()
# X_test = X_test_orig.copy()

# --- 2. Preprocessing (Illustrative - adapt to your 160 columns) ---
# Identify numerical and categorical columns (example subset)
# numerical_cols = X_train.select_dtypes(include=np.number).columns.tolist()
# categorical_cols = ['race'] # Add other categoricals

# Create preprocessing pipelines for numerical and categorical data
# numerical_pipeline = Pipeline([
#     ('imputer', SimpleImputer(strategy='median')),
#     ('scaler', StandardScaler())
# ])
#
# categorical_pipeline = Pipeline([
#     ('imputer', SimpleImputer(strategy='most_frequent')), # or 'constant', fill_value='Missing'
#     ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
# ])

# Combine preprocessing pipelines using ColumnTransformer
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', numerical_pipeline, numerical_cols),
#         ('cat', categorical_pipeline, categorical_cols)
#     ],
#     remainder='passthrough' # Or 'drop' if unlisted columns are not needed
# )

# --- 3. Model Training with Strategies ---

# Strategy A: Cost-Sensitive Learning for Target Imbalance
# model_cost_sensitive = LogisticRegression(solver='liblinear', class_weight='balanced', random_state=42)
# pipeline_cost_sensitive = Pipeline([
#     ('preprocessor', preprocessor),
#     ('classifier', model_cost_sensitive)
# ])
# # pipeline_cost_sensitive.fit(X_train, y_train)
# print(f"Training with Cost-Sensitive Learning for y. Original y_train counts: {Counter(y_train)}")

# Strategy B: SMOTE for Target Imbalance (using imblearn's Pipeline)
# model_smote = LogisticRegression(solver='liblinear', random_state=42) # No class_weight here
# pipeline_smote = ImbPipeline([
#     ('preprocessor', preprocessor),
#     ('sampler', SMOTE(random_state=42, sampling_strategy='auto')), # Default is to oversample minority
#     ('classifier', model_smote)
# ])
# # pipeline_smote.fit(X_train, y_train)
# # Access post-SMOTE counts conceptually (sampler is part of pipeline)
# # X_train_processed_temp = preprocessor.fit_transform(X_train)
# # y_train_smote_temp = SMOTE(random_state=42).fit_resample(X_train_processed_temp, y_train)[1]
# # print(f"Training with SMOTE for y. y_train counts after SMOTE (conceptual): {Counter(y_train_smote_temp)}")


# Strategy C: Sample Re-weighting for Feature Imbalance (e.g., 'race')
# This is applied via sample_weight in model.fit()
# model_reweighted = LogisticRegression(solver='liblinear', random_state=42) # Potentially also class_weight='balanced' for target
# Fit preprocessor first
# X_train_processed = preprocessor.fit_transform(X_train)
# X_test_processed = preprocessor.transform(X_test) # Transform test set too

# Create sample weights (conceptual - based on original X_train before processing if needed for feature value)
# Assuming 'race' column is still accessible or mapped from preprocessed data
# race_weights_map = {'white': 1.0, 'black': len(X_train) / X_train['race'].value_counts()['black']} # Example weights
# sample_weights = X_train['race'].map(race_weights_map).fillna(1.0).to_numpy()
#
# # If using class_weight='balanced' with sample_weight, the effects multiply.
# # model_reweighted.fit(X_train_processed, y_train, sample_weight=sample_weights)
# print(f"Training with sample re-weighting for 'race'. (Example weights: {race_weights_map})")


# --- 4. Evaluation (on X_test_processed, y_test) ---
# For each trained pipeline/model:
# y_pred = pipeline_X.predict(X_test)
# y_proba = pipeline_X.predict_proba(X_test)[:, 1]
#
# Calculate overall performance (AUC, F1 for 'Enrolled', etc.)
# Calculate fairness metrics (Demographic Parity, Equal Opportunity, etc.)
# by comparing y_pred/y_proba to y_test, disaggregated by columns in X_test_orig
# (e.g., X_test_orig['race'], X_test_orig['dem_female'], etc.)


## comment
adapt the loading/splitting lines (define df, X, y).

Define numerical_cols (e.g. df.select_dtypes(include=np.number)) and list your categorical columns (e.g. ['race', 'dem_female', …]).

Install and import imbalanced-learn (SMOTE, ImbPipeline).

Instantiate and fit each pipeline/model (you’ll need to call .fit() and then .predict() or .predict_proba() for each strategy).

Compute the actual fairness metrics—e.g. false-positive rates per group—after predicting on your held-out test set.

Handle any missing values or encoding issues exactly as in your working pipeline.

## Below the new code:


In [38]:
import pandas as pd
import numpy as np
from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.metrics import accuracy_score, roc_auc_score

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

# 1. Load data
df = pd.read_csv("data_new.csv")

# 2. Define target and features
y = df["program_enrolled_t"]                     # outcome at time t :contentReference[oaicite:1]{index=1}
X = df.drop(columns=["program_enrolled_t"])

# 3. Train/test split (stratify on y for balanced classes)
X_train_orig, X_test_orig, y_train, y_test = train_test_split(
    X, y,
    test_size=0.25,
    stratify=y,
    random_state=42
)

# Working copies
X_train = X_train_orig.copy()
X_test  = X_test_orig.copy()

# 4. Identify numerical and categorical columns
numerical_cols   = X_train.select_dtypes(include=["int64", "float64"]).columns.tolist()
categorical_cols = ["race", "dem_female"]        # both exist in data_new.csv :contentReference[oaicite:2]{index=2}

# 5. Preprocessing pipelines
numerical_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler",   StandardScaler())
])
categorical_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot",  OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numerical_pipeline,   numerical_cols),
        ("cat", categorical_pipeline, categorical_cols)
    ],
    remainder="drop"
)

# 6A. Strategy A: Cost‐Sensitive Learning for y imbalance
model_cost = LogisticRegression(
    solver="liblinear",
    class_weight="balanced",
    random_state=42
)
pipeline_cost = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier",   model_cost)
])
pipeline_cost.fit(X_train, y_train)
y_pred_cost  = pipeline_cost.predict(X_test)
y_proba_cost = pipeline_cost.predict_proba(X_test)[:, 1]
print("Cost‐Sensitive")
print("y_train counts:", Counter(y_train))
print("Accuracy:",  accuracy_score(y_test, y_pred_cost))
print("ROC-AUC:",   roc_auc_score(y_test, y_proba_cost))
print()

# 6B. Strategy B: SMOTE Oversampling for y imbalance
model_smote = LogisticRegression(solver="liblinear", random_state=42)
pipeline_smote = ImbPipeline([
    ("preprocessor", preprocessor),
    ("smote",        SMOTE(random_state=42, sampling_strategy="auto")),
    ("classifier",   model_smote)
])
pipeline_smote.fit(X_train, y_train)
y_pred_smote  = pipeline_smote.predict(X_test)
y_proba_smote = pipeline_smote.predict_proba(X_test)[:, 1]
print("SMOTE Oversampling")
print("y_train counts before SMOTE:", Counter(y_train))
print("Accuracy:",  accuracy_score(y_test, y_pred_smote))
print("ROC-AUC:",   roc_auc_score(y_test, y_proba_smote))
print()

# 6C. Strategy C: Sample Re‐weighting for 'race'
race_counts    = X_train["race"].value_counts()                  # white/black :contentReference[oaicite:3]{index=3}
race_weights   = {lvl: len(X_train) / cnt for lvl, cnt in race_counts.items()}
sample_weights = X_train["race"].map(race_weights).fillna(1.0).to_numpy()

model_reweight = LogisticRegression(solver="liblinear", random_state=42)
pipeline_rew = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier",   model_reweight)
])
pipeline_rew.fit(
    X_train,
    y_train,
    classifier__sample_weight=sample_weights
)
y_pred_rew  = pipeline_rew.predict(X_test)
y_proba_rew = pipeline_rew.predict_proba(X_test)[:, 1]
print("Sample Re‐weighting (race)")
print("race counts:", race_counts.to_dict())
print("Sample weights map:", race_weights)
print("Accuracy:",  accuracy_score(y_test, y_pred_rew))
print("ROC-AUC:",   roc_auc_score(y_test, y_proba_rew))


=== Cost‐Sensitive ===
y_train counts: Counter({0: 36249, 1: 339})
Accuracy: 0.8510987208920958
ROC-AUC: 0.8855819519708445

=== SMOTE Oversampling ===
y_train counts before SMOTE: Counter({0: 36249, 1: 339})
Accuracy: 0.8415874057067891
ROC-AUC: 0.8644757243226973

=== Sample Re‐weighting (race) ===
race counts: {'white': 32391, 'black': 4197}
Sample weights map: {'white': 1.129573029545244, 'black': 8.717655468191566}
Accuracy: 0.9897507379468679
ROC-AUC: 0.8072007845440717


## Output:
The cost sensitive approach which heavily penalizes misclassifying the minority class reduced overall accuracy to about 85.1 percent and yielded a ROC AUC of 0.886. SMOTE oversampling produced similar effects, with accuracy of 84.2 percent and ROC AUC of 0.864 as synthetic minority examples encouraged the model to predict more positives. Finally, reweighting samples by race so that each white case carried a weight of approximately 1.13 and each black case a weight of about 8.72 restored very high accuracy of 98.98 percent while maintaining a ROC AUC of 0.807. This shows that placing greater emphasis on under represented groups can preserve the model’s ability to distinguish true positives from true negatives without undermining its overall discrimination performance.